In [1]:
import angr
import simuvex
import logging

# DEFCON - BABY-RE
# @author: P1kachu
# @contact: p1kachu@lse.epita.fr

In [2]:
p = angr.Project('step')

In [3]:
win            = 0x400e0b  # good
fail           = 0x400df2  # fail
main           = 0x400fa6  # Address of main
find           = (win,)
avoid          = (fail,)

In [4]:
def patch_fgets(state):
    size = state.regs.rsi.ast.args[0]
    buff = state.regs.rdi.ast.args[0]
    print("fgets {0} bytes at {1}".format(size, hex(buff)))
    for x in xrange(size):
        state.mem[buff + x:] = state.se.BVS('pass', 8)

In [5]:
init = p.factory.blank_state(addr=main)

# Patch fgets
p.hook(0x400fee, func=patch_fgets, length=5)

In [6]:
pgp = p.factory.path_group(init, threads=8)

# Now stuff becomes interesting
ex = pgp.explore(find=find, avoid=avoid)

print(ex.found)
s = ex.found[0].state

fgets 6 bytes at 0x7fffffffffeffe8L
[<Path with 170 runs (at 0x400e0b)>]


In [36]:
tmp = s.memory.load(0x7fffffffffeffe8, 1)
print(tmp.args)
print("The flag is '{0}'".format(tmp))

('pass_1_8', None, None, None, False, False, None)
The flag is '<BV8 pass_1_8>'
